In [ ]:
# from docs.test.zwplib import LakeHouse

# import liblakehouse
from liblakehouse import LakeHouse

In [ ]:
lakehouse = LakeHouse()

In [ ]:
# landing tier
import requests

url = 'https://servicodados.ibge.gov.br/api/v3/agregados/7060/periodos/202001/variaveis/63|69|2265|66?localidades=N1[all]|N6[all]&classificacao=315[all]'

response = requests.get(url)

with open('ipca_amplo.json', 'wb') as bf:
    bf.write(response.content)


In [ ]:
lakehouse.write_files(catalog='landing', schema='ibge', volume='ipca_amplo', files=['ipca_amplo.json'])

In [ ]:
# bronze tier
df = lakehouse.read_files(
    catalog="landing",
    schema="ibge",
    volume="ipca_amplo",
    files=[
        {
            "path": "ipca_amplo.json",
            "format": "json",
            "schema": "",
            "options": {},
        }
    ],
)

In [ ]:
df[0].show()

In [ ]:
lakehouse.write_table(
    catalog="bronze",
    schema="ibge",
    table="ipca_amplo",
    dataframe={
        "dataframe": df[0],
        "format": "delta",
        "mode": "overwrite",
        "partition_by": [],
        "options": {"overwriteSchema": "true"},
    },
)

In [ ]:
# silver tier
df = lakehouse.read_table(
    catalog="bronze",
    schema="ibge",
    table="ipca_amplo",
    options={
        "format": "delta",
        "options": {}
    }    
)

In [ ]:
df.show()

In [ ]:
lakehouse.write_table(
    catalog="silver",
    schema="ibge",
    table="ipca_amplo",
    dataframe={
        "dataframe": df,
        "format": "delta",
        "mode": "overwrite",
        "partition_by": [],
        "options": {"overwriteSchema": "true"},
    },
)

In [ ]:
# 25/08/21 07:54:12 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
# SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
# SLF4J: Defaulting to no-operation (NOP) logger implementation
# SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.

# 25/08/21 07:54:15 WARN HiveClientImpl: Detected HiveConf hive.execution.engine is 'tez' and will be reset to 'mr' to disable useless hive logic
# Hive Session ID = e38db77a-6afe-437e-bc8e-d5fa7f632b38
# 25/08/21 07:54:43 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
# 25/08/21 07:54:51 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to numbers/integer/part-00000-8c5f40d3-4def-4580-b432-fc63efb209cc-c000.snappy.parquet. This is Unsupported
# 25/08/21 07:55:02 WARN HiveExternalCatalog: Could not alter schema of table `numbers`.`integer` in a Hive compatible way. Updating Hive metastore in Spark SQL specific format.

# Caused by: org.apache.spark.rpc.RpcEndpointNotFoundException: Cannot find endpoint: spark://CoarseGrainedScheduler@10.255.255.254:34253

# WARN HiveClientImpl: Detected HiveConf hive.execution.engine is 'tez' and will be reset to 'mr' to disable useless hive logic

In [ ]:
# create catalog bronze using delta_lake with (
#     "hive.metastore.uri"='thrift://host.docker.internal:9083',
#     "s3.aws-access-key"='9a5edc7df42cfd71',
#     "s3.aws-secret-key"='DW4CyCit2CGO9gXyRtqQ7Lj6ZzQZm3xfS33ItBQpVbc=',
#     "s3.endpoint"='http://host.docker.internal:10000',
#     "s3.region"='us-east-1',
#     "fs.native-s3.enabled"='true',
#     "s3.path-style-access"='true',
#     "delta.metastore.store-table-metadata"='true',
#     "delta.register-table-procedure.enabled"='true',
#     "delta.deletion-vectors-enabled"='true',
#     "delta.enable-non-concurrent-writes"='true'
# )

# create schema if not exists number with (
#     "location" = 's3a://bronze/numbers'
# )

# CALL bronze.system.register_table(schema_name => 'numbers', table_name => 'integers', table_location => 's3://bronze/numbers/integergs')

In [ ]:
# Creating a Git Commit Message Convention for Your Team

# https://medium.com/@naandalist/creating-a-git-commit-message-convention-for-your-team-acb4b3edfc44